In [30]:
# Set up modules for Google functionality
from google.cloud import bigquery # To run BQ statements
from google_auth_oauthlib import flow # To authorise as user
from googleapiclient.discovery import build # To pull in from sheets, slides etc. API
from google.auth.transport.requests import Request

# Display
import pprint

# Operating system stuff
import pickle
import os.path
import sys

# Data handling
import json
import requests
from pandas import read_csv
import zipfile
import io
import numpy as np

# Stats, models, datasheets
import pandas as pd
import pyreadstat

# Visualisation
from matplotlib import pyplot
import matplotlib.pyplot as plt
import matplotlib_venn # For venn diagrams
from pandas.plotting import autocorrelation_plot

# Network graphs
import networkx as nx

# Misc
from xlsxwriter.utility import xl_rowcol_to_cell # Used to create cell references
import itertools

# Load custom scripts in reusable_code folder
sys.path.append(r'/home/jupyter/reusable_code')

import google_api_functions as gaf
import my_qualtrics as qltx

In [12]:
my_session=qltx.session('ExTDN3nBP2Q8vVVnUJeMsjUCbyuWPyEDxblfUqCW')

In [13]:
#my_session.Get_list_of_surveys()

In [14]:
joiners=my_session.Get_survey_id_from_name('Joiners survey')
joiners                               

In [15]:
#my_session.Get_Account_Info('britboxresearch')

In [47]:
joiners='SV_9TrvZkZsOeTYy8t' # This is the Editorial Concept Testing- which was more complex

In [48]:
survey_results=my_session.Download_survey_responses(joiners,from_time='2020-03-05T00:00:00Z',to_time='2020-03-07T11:53:34Z')
print(survey_results['StartDate'].min(),survey_results['StartDate'].max())

In [49]:
survey_results=my_session.Download_survey_responses(joiners)
print(survey_results['StartDate'].min(),survey_results['StartDate'].max())

In [50]:
survey_results.head()

### Create a flag to indicate "shown" and then insert missing values for all the "shown but not answered" questions.

In [51]:
Qlist=[i for i in survey_results.columns if i[0]=='Q'] # List of questions
Asked_list=["Asked_"+i for i in Qlist]# List of questions prefixed with "Asked_"

# Creates a binary flag prefixed "Asked_" for each Q to indicate whether it was shown
survey_results[Asked_list] = survey_results[Qlist].where(survey_results[Qlist].isnull(), 1).fillna(0).astype(int)

# Replace the -999 returned by the API call with a null
survey_results[Qlist]=survey_results[Qlist].replace('-999',np.NaN)
survey_results.head()

In [52]:
survey_results.shape

We need to handle the difference between questions not asked because of routing and questions skipped by respondents.
Those not asked are blank, those asked but skipped will be -999 by default.
https://api.qualtrics.com/reference#create-response-export-new


In [65]:
survey_info=my_session.Get_Survey_Info(joiners)
survey_q_map=survey_info['questions']
survey_q_map_df=pd.DataFrame.from_dict(survey_q_map,orient='index')

In [73]:
from collections import MutableMapping 
  
# code to convert ini_dict to flattened dictionary 
# default seperater '_' 
def convert_flatten(d, parent_key ='', sep ='_'): 
    items = [] 
    for k, v in d.items(): 
        new_key = parent_key + sep + k if parent_key else k 
  
        if isinstance(v, MutableMapping): 
            items.extend(convert_flatten(v, new_key, sep = sep).items()) 
        else: 
            items.append((new_key, v)) 
    return dict(items) 

survey_q_map_flat=[convert_flatten(survey_q_map[q]) for q in survey_q_map]

In [80]:
pd.DataFrame.from_dict(survey_q_map_flat)

In [64]:
qmap=[]
for q in survey_q_map:
    qdict={}
    qdict['type']=survey_q_map[q]['questionType']['type']
    qdict['selector']=survey_q_map[q]['questionType']['selector']
    qdict['subSelector']=survey_q_map[q]['questionType']['subSelector']
    qdict['questionText']=survey_q_map[q]['questionText']
    qdict['questionLabel']=survey_q_map[q]['questionLabel']
    qdict['validation']=survey_q_map[q]['validation']
    qdict['questionName']=survey_q_map[q]['questionName']
    
    try:
        print("SubQs are...")
        print(survey_q_map[q]['subQuestions'].keys())
    except:
        pass
    
    try:
        print("Choices are...")
        print(survey_q_map[q]['subQuestions'].keys())
    exce
    
    print(survey_q_map[q].keys())
    #print(survey_q_map[q])
    